In [5]:
import requests
import re
import datetime
import random
from time import sleep
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from pandas import Series, DataFrame
import sqlalchemy
from sqlalchemy import create_engine,Table,Column,Integer,String,MetaData,ForeignKey,Date,update
import warnings
from IPython.display import display

In [6]:
def load_csv(date):
    df=pd.DataFrame.from_csv("data/reviews-%s.csv"%(date), encoding="utf-8",index_col=None)
    #.reset_index(drop=True)
    return df

In [7]:
def summary(df):
    prodTypes=["central","canister","handheld","robotic","stick","upright","wetdry"]
    #choose type
    for prodType in prodTypes:
        print(prodType,len(df.loc[df["ptype"]==prodType]["review"]))
    print("total reviews=",len(df),"\n")

In [8]:
def store_prodRevs_in_a_list(prodType,df):
    df=df.loc[df["ptype"]==prodType]
    pReviewsGroup=df.groupby(['ptype','pid'])

    reviews_list=[]
    for key,pReviews in pReviewsGroup:
        rids=pReviews[["rid","review"]].values[:,0].tolist()
        reviews=pReviews[["rid","review"]].values[:,1].tolist()

        ptypes=[key[0]]*len(rids)
        pids=[key[1]]*len(rids)
        reviews_list+=list(zip(ptypes,pids,rids,reviews))

    print("number of products=",len(pReviewsGroup))
    print("number of reviews in type %s=%i"%(prodType,len(reviews_list)))
    
    return reviews_list

In [9]:
#product types
prodTypes=["central","canister","handheld","robotic","stick","upright","wetdry"]

In [10]:
df=load_csv("2017-02-03")
summary(df)

reviews_list=store_prodRevs_in_a_list(prodTypes[0],df)

central 335
canister 23294
handheld 79899
robotic 26944
stick 57268
upright 102643
wetdry 16821
total reviews= 307204 

number of products= 17
number of reviews in type central=335


In [89]:
reviews_list

[('central',
  'B001VS8WLE',
  249888,
  "This product really sucks! I could't resist the juvenile pun. Recently installed one of these as a replacement for an older vac unit that bit the dust. 4,000 square foot home with 5 inlets. Works great. Plenty of suction power, easy to install and easy to empty. Great price too. Two thumbs up for a mid range central vac unit."),
 ('central',
  'B001VS8WLE',
  249889,
  'Here\'s the problem - I ordered this system to replace an existing unit that was old and had worn out. The first unit I received didn\'t even power up. Amazon\'s great and shipped a replacement asap.The replacement worked great ... for a while. About a month ago (after only 9 months of service) the unit began just shutting down in the middle of vacuuming usually after 2-3 minutes of use. Required a hard reset - as in unplug and replug in. Since it\'s in a corner of the basement, that\'s a little inconvenient! Amazon says the return period is over and I have to contact the manufa

In [72]:
#exercise:re.sub

#https://docs.python.org/3/library/re.html

text = "Professor Abd.....olmalek..., please report your absences promptly..."
#print(re.match(r"(\w)(\w+)(\w)",text).group(1))
re.sub("\.+", ".",text)

'Professor Abd.olmalek., please report your absences promptly.'

In [84]:
text="hahaha:)   :(     "
re.sub('(?::|;|=)(?:-)?(?:\)|\(|D|P)', ".",text)

'hahaha.   .     '

In [87]:
def cleanText(review):
    review=review.lower().replace('\n','').replace(';',',').replace('!',',')
    review=re.sub("\.+",".", review)
    
    #remove emoji
    emoji_pattern = re.compile("["  
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    review=re.sub(emoji_pattern,"", review)
    review=re.sub('(?::|;|=)(?:-)?(?:\)|\(|D|P)',"",review)
    
    
    return review

In [88]:
for review in reviews_list:
    print(cleanText(review[3]))

this product really sucks, i could't resist the juvenile pun. recently installed one of these as a replacement for an older vac unit that bit the dust. 4,000 square foot home with 5 inlets. works great. plenty of suction power, easy to install and easy to empty. great price too. two thumbs up for a mid range central vac unit.
here's the problem - i ordered this system to replace an existing unit that was old and had worn out. the first unit i received didn't even power up. amazon's great and shipped a replacement asap.the replacement worked great . for a while. about a month ago (after only 9 months of service) the unit began just shutting down in the middle of vacuuming usually after 2-3 minutes of use. required a hard reset - as in unplug and replug in. since it's in a corner of the basement, that's a little inconvenient, amazon says the return period is over and i have to contact the manufacturer. so i call honeywell and they refer me to company xxx for warranty service in my area. 

In [21]:
a=['aa|a','bb|b']

def f(x):
    return x.split("|")
 

list(map(f,a))

[['aa', 'a'], ['bb', 'b']]

In [22]:
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n','') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

In [26]:
text=["This vacuum is a great value. dsadsa"]
cleanText(text)

[['this', 'vacuum', 'is', 'a', 'great', 'value', '.', 'dsadsa']]